# 🌍 Překlad a vícejazyčné modely

**Autor:** Praut s.r.o. - AI Integration & Business Automation

## Co se naučíte:
- Automatický překlad mezi jazyky
- Vícejazyčné embedding modely
- Lokalizace obsahu
- Cross-lingual aplikace

In [ ]:
!pip install -q transformers accelerate torch sentencepiece sacremoses

In [ ]:
from transformers import pipeline, MarianMTModel, MarianTokenizer
import torch

device = 0 if torch.cuda.is_available() else -1
print(f"🖥️ Device: {'GPU' if device == 0 else 'CPU'}")

## 1. Základní překlad s MarianMT

In [ ]:
# Angličtina -> Čeština
en_cs = pipeline("translation", model="Helsinki-NLP/opus-mt-en-cs", device=device)

anglicke_texty = [
    "Artificial intelligence is transforming business processes.",
    "Our company specializes in automation solutions.",
    "Machine learning models can analyze large datasets quickly."
]

print("🇬🇧 → 🇨🇿 Překlad angličtina -> čeština:\n")
for text in anglicke_texty:
    preklad = en_cs(text)
    print(f"EN: {text}")
    print(f"CS: {preklad[0]['translation_text']}\n")

In [ ]:
# Čeština -> Angličtina
cs_en = pipeline("translation", model="Helsinki-NLP/opus-mt-cs-en", device=device)

ceske_texty = [
    "Naše společnost nabízí komplexní řešení pro automatizaci.",
    "Umělá inteligence pomáhá firmám šetřit čas i peníze.",
    "Vývoj software na míru je naší specializací."
]

print("🇨🇿 → 🇬🇧 Překlad čeština -> angličtina:\n")
for text in ceske_texty:
    preklad = cs_en(text)
    print(f"CS: {text}")
    print(f"EN: {preklad[0]['translation_text']}\n")

## 2. Multijazyčný překlad (NLLB)

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# NLLB (No Language Left Behind) - podporuje 200+ jazyků
model_name = "facebook/nllb-200-distilled-600M"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

if device == 0:
    model = model.to("cuda")

def preloz_nllb(text, src_lang, tgt_lang):
    """Přeloží text mezi libovolnými jazyky."""
    tokenizer.src_lang = src_lang
    inputs = tokenizer(text, return_tensors="pt")
    
    if device == 0:
        inputs = {k: v.to("cuda") for k, v in inputs.items()}
    
    generated = model.generate(
        **inputs,
        forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_lang),
        max_length=256
    )
    
    return tokenizer.decode(generated[0], skip_special_tokens=True)

# Jazykové kódy NLLB
# ces_Latn = čeština, eng_Latn = angličtina, deu_Latn = němčina, 
# fra_Latn = francouzština, spa_Latn = španělština

text = "Welcome to our company. We specialize in AI solutions."

print(f"📝 Originál (EN): {text}\n")
print("🌍 Překlady do různých jazyků:")

jazyky = [
    ("ces_Latn", "🇨🇿 Čeština"),
    ("deu_Latn", "🇩🇪 Němčina"),
    ("fra_Latn", "🇫🇷 Francouzština"),
    ("spa_Latn", "🇪🇸 Španělština"),
]

for lang_code, lang_name in jazyky:
    preklad = preloz_nllb(text, "eng_Latn", lang_code)
    print(f"   {lang_name}: {preklad}")

## 3. Hromadný překlad dokumentů

In [ ]:
import pandas as pd

# Simulace katalogu produktů
produkty = pd.DataFrame({
    "id": [1, 2, 3],
    "nazev_cs": ["Automatizační software", "AI asistent", "Analytická platforma"],
    "popis_cs": [
        "Komplexní řešení pro automatizaci firemních procesů.",
        "Inteligentní asistent poháněný umělou inteligencí.",
        "Platforma pro analýzu dat a business intelligence."
    ]
})

# Překlad do angličtiny
produkty["nazev_en"] = [cs_en(t)[0]['translation_text'] for t in produkty["nazev_cs"]]
produkty["popis_en"] = [cs_en(t)[0]['translation_text'] for t in produkty["popis_cs"]]

print("📦 Lokalizovaný katalog produktů:\n")
for _, row in produkty.iterrows():
    print(f"ID: {row['id']}")
    print(f"🇨🇿 {row['nazev_cs']}: {row['popis_cs']}")
    print(f"🇬🇧 {row['nazev_en']}: {row['popis_en']}")
    print()

## 4. Cross-lingual sentiment analýza

In [ ]:
# Vícejazyčný sentiment model
multilang_sentiment = pipeline(
    "sentiment-analysis",
    model="nlptown/bert-base-multilingual-uncased-sentiment",
    device=device
)

# Recenze v různých jazycích
recenze = [
    ("🇬🇧", "This product is amazing, I love it!"),
    ("🇨🇿", "Skvělý produkt, naprostá spokojenost!"),
    ("🇩🇪", "Sehr gut! Ich bin sehr zufrieden."),
    ("🇫🇷", "Produit de mauvaise qualité, je suis déçu."),
    ("🇪🇸", "Excelente servicio, muy recomendable.")
]

print("🌍 Vícejazyčná sentiment analýza:\n")
for vlajka, text in recenze:
    result = multilang_sentiment(text)[0]
    stars = int(result['label'].split()[0])
    print(f"{vlajka} {'⭐' * stars}{'☆' * (5-stars)} ({result['score']:.1%})")
    print(f"   \"{text}\"\n")

## 5. Automatická detekce jazyka

In [ ]:
# Detekce jazyka
lang_detector = pipeline("text-classification", 
                         model="papluca/xlm-roberta-base-language-detection",
                         device=device)

texty = [
    "Hello, how can I help you today?",
    "Dobrý den, jak vám mohu pomoci?",
    "Guten Tag, wie kann ich Ihnen helfen?",
    "Bonjour, comment puis-je vous aider?",
    "Hola, ¿cómo puedo ayudarle?"
]

print("🔍 Automatická detekce jazyka:\n")
for text in texty:
    result = lang_detector(text)[0]
    print(f"   [{result['label']}] ({result['score']:.1%}): {text}")

## 6. Automatický překladač s detekcí jazyka

In [ ]:
# Mapování jazyků na NLLB kódy
LANG_MAP = {
    "en": "eng_Latn",
    "cs": "ces_Latn",
    "de": "deu_Latn",
    "fr": "fra_Latn",
    "es": "spa_Latn",
    "it": "ita_Latn",
    "pl": "pol_Latn",
    "sk": "slk_Latn"
}

def auto_preklad(text, cilovy_jazyk="cs"):
    """Automaticky detekuje jazyk a přeloží do cílového jazyka."""
    
    # Detekce zdrojového jazyka
    detected = lang_detector(text)[0]
    src_lang = detected['label']
    
    # Pokud je text již v cílovém jazyce
    if src_lang == cilovy_jazyk:
        return {
            "original": text,
            "translated": text,
            "source_lang": src_lang,
            "target_lang": cilovy_jazyk,
            "note": "Text je již v cílovém jazyce"
        }
    
    # Překlad pomocí NLLB
    src_code = LANG_MAP.get(src_lang, "eng_Latn")
    tgt_code = LANG_MAP.get(cilovy_jazyk, "ces_Latn")
    
    preklad = preloz_nllb(text, src_code, tgt_code)
    
    return {
        "original": text,
        "translated": preklad,
        "source_lang": src_lang,
        "target_lang": cilovy_jazyk
    }

# Test
test_texty = [
    "Welcome to our service portal.",
    "Willkommen bei unserem Serviceportal.",
    "Bienvenue sur notre portail de services."
]

print("🔄 Automatický překlad do češtiny:\n")
for text in test_texty:
    vysledek = auto_preklad(text, "cs")
    print(f"[{vysledek['source_lang']}] {vysledek['original']}")
    print(f"[{vysledek['target_lang']}] {vysledek['translated']}\n")

## 7. Lokalizace UI textů

In [ ]:
import json

# Zdrojové UI texty v angličtině
ui_texty_en = {
    "welcome": "Welcome to our application",
    "login": "Please log in to continue",
    "dashboard": "Dashboard",
    "settings": "Settings",
    "logout": "Log out",
    "save": "Save changes",
    "cancel": "Cancel",
    "error": "An error occurred. Please try again."
}

def lokalizuj_ui(texty, cilovy_jazyk):
    """Lokalizuje UI texty do cílového jazyka."""
    lokalizovane = {}
    
    for klic, text in texty.items():
        tgt_code = LANG_MAP.get(cilovy_jazyk, "ces_Latn")
        lokalizovane[klic] = preloz_nllb(text, "eng_Latn", tgt_code)
    
    return lokalizovane

# Generování lokalizací
print("🌐 Lokalizace UI textů:\n")

ui_cs = lokalizuj_ui(ui_texty_en, "cs")
print("🇨🇿 Čeština:")
print(json.dumps(ui_cs, indent=2, ensure_ascii=False))

## 8. Překladový workflow pro dokumenty

In [ ]:
def preloz_dokument(dokument, cilovy_jazyk="cs"):
    """Přeloží celý dokument po odstavcích."""
    
    # Rozdělení na odstavce
    odstavce = [p.strip() for p in dokument.split('\n') if p.strip()]
    
    prelozene = []
    for i, odstavec in enumerate(odstavce):
        vysledek = auto_preklad(odstavec, cilovy_jazyk)
        prelozene.append(vysledek['translated'])
        print(f"   ✓ Odstavec {i+1}/{len(odstavce)}")
    
    return '\n\n'.join(prelozene)

# Test dokument
dokument = """
Welcome to our AI automation platform.

Our solution helps businesses streamline their operations through artificial intelligence.

Key features include automated document processing, intelligent customer support, and predictive analytics.
"""

print("📄 Překládám dokument...\n")
prelozeny = preloz_dokument(dokument, "cs")

print("\n📝 Přeložený dokument:")
print(prelozeny)

---
## 🏁 Shrnutí

- ✅ MarianMT pro rychlé překlady mezi jazykovými páry
- ✅ NLLB pro 200+ jazyků
- ✅ Automatická detekce jazyka
- ✅ Vícejazyčná sentiment analýza
- ✅ Lokalizace UI a dokumentů

**Další notebook:** Question Answering a vyhledávání v dokumentech